# Desafío - Spark I

**Nombre:** Julio Valdés
<br>
**Generación:** GC

### Ejercicio 1: Preliminares y datos perdidos

#### Genere los objetos SparkConf y SparkContext

In [18]:
from pyspark import SparkConf, SparkContext
spark_conf = SparkConf()\
            .setMaster("local[4]")\
            .setAppName('desafio-spark-I')
spark_conf

In [19]:
sc

<SparkContext master=yarn appName=livy-session-0>

#### Utilizando Spark , importe el archivo household_power_consumption.txt a un objeto RDD

In [20]:
hpc_data = sc.textFile('s3://bigdata-desafio/challenges/u4lec1/household_power_consumption.txt')
hpc_data.take(5)

['Date;Time;Global_active_power;Global_reactive_power;Voltage;Global_intensity;Sub_metering_1;Sub_metering_2;Sub_metering_3', '16/12/2006;17:24:00;4.216;0.418;234.840;18.400;0.000;1.000;17.000', '16/12/2006;17:25:00;5.360;0.436;233.630;23.000;0.000;1.000;16.000', '16/12/2006;17:26:00;5.374;0.498;233.290;23.000;0.000;2.000;17.000', '16/12/2006;17:27:00;5.388;0.502;233.740;23.000;0.000;1.000;17.000']

#### El primer registro del archivo corresponde a las etiquetas asociadas a cada columna. Elimínelo de su RDD.

In [21]:
header = hpc_data.first()
hpc_proc = hpc_data\
            .filter(lambda x: x != header)
hpc_proc.take(5)

['16/12/2006;17:24:00;4.216;0.418;234.840;18.400;0.000;1.000;17.000', '16/12/2006;17:25:00;5.360;0.436;233.630;23.000;0.000;1.000;16.000', '16/12/2006;17:26:00;5.374;0.498;233.290;23.000;0.000;2.000;17.000', '16/12/2006;17:27:00;5.388;0.502;233.740;23.000;0.000;1.000;17.000', '16/12/2006;17:28:00;3.666;0.528;235.680;15.800;0.000;1.000;17.000']

#### Cada campo dentro del registro está separado por un ; . Separe cada campo del registro por éste caracter.

In [22]:
hpc_proc = hpc_data\
            .filter(lambda x: x != header)\
            .map(lambda x: x.split(';'))\
            .map(lambda x: (str(x[0]), str(x[1]), float(x[2]), float(x[3]), float(x[4]), 
                            float(x[5]), float(x[6]), float(x[7]), float(x[8])))
hpc_proc.take(5)

[('16/12/2006', '17:24:00', 4.216, 0.418, 234.84, 18.4, 0.0, 1.0, 17.0), ('16/12/2006', '17:25:00', 5.36, 0.436, 233.63, 23.0, 0.0, 1.0, 16.0), ('16/12/2006', '17:26:00', 5.374, 0.498, 233.29, 23.0, 0.0, 2.0, 17.0), ('16/12/2006', '17:27:00', 5.388, 0.502, 233.74, 23.0, 0.0, 1.0, 17.0), ('16/12/2006', '17:28:00', 3.666, 0.528, 235.68, 15.8, 0.0, 1.0, 17.0)]

### Reporte las siguientes métricas:

#### La cantidad total de registros antes de eliminar los datos perdidos.

In [23]:
print("Cantidad total de registros antes de eliminar los datos perdidos: {}".format(hpc_data.count()))

Cantidad total de registros antes de eliminar los datos perdidos: 2075260

#### La cantidad total de registros nulos.

In [24]:
hpc_proc = hpc_data\
            .filter(lambda x: '?' in x)
print("Cantidad total de registros nulos: {}".format(hpc_proc.count()))

Cantidad total de registros nulos: 25979

#### La cantidad de registros nulos por año. ¿En qué año hubo más registros nulos?

In [25]:
hpc_proc = hpc_data\
            .filter(lambda x: '?' in x)\
            .map(lambda x: x.split(';'))\
            .map(lambda x: (x[0].split('/')[2], 1))\
            .reduceByKey(lambda x, y: x + y)\
            .sortBy(lambda x: -x[1])
result = hpc_proc.collect()
print(result)
print("Hubo más registros nulos en el año {}".format(result[0][0]))

[('2010', 17629), ('2009', 4280), ('2007', 3931), ('2008', 135), ('2006', 4)]
Hubo más registros nulos en el año 2010

#### La cantidad de registros nulos por mes para el año con una mayor cantidad de registros nulos. ¿En qué mes hubo más registros nulos?

In [26]:
hpc_proc = hpc_data\
            .filter(lambda x: '?' in x)\
            .map(lambda x: x.split(';'))\
            .map(lambda x: (x[0].split('/')[2], x[0].split('/')[1], 1))\
            .filter(lambda x: x[0] == '2010')\
            .map(lambda x: (x[1], 1))\
            .reduceByKey(lambda x, y: x + y)\
            .sortBy(lambda x: -x[1])
result = hpc_proc.collect()
print(result)
print("Hubo más registros nulos en el mes {}".format(result[0][0]))

[('8', 7226), ('9', 5237), ('1', 3131), ('3', 2027), ('2', 2), ('6', 2), ('4', 1), ('10', 1), ('5', 1), ('7', 1)]
Hubo más registros nulos en el mes 8

### Ejercicio 2: Preparación de los datos

#### Genere un objeto donde se encuentren sólo aquellos datos sin registros perdidos.

In [27]:
hpc_proc = hpc_data\
            .filter(lambda x: x != header)\
            .filter(lambda x: '?' not in x)
hpc_proc.take(5)

['16/12/2006;17:24:00;4.216;0.418;234.840;18.400;0.000;1.000;17.000', '16/12/2006;17:25:00;5.360;0.436;233.630;23.000;0.000;1.000;16.000', '16/12/2006;17:26:00;5.374;0.498;233.290;23.000;0.000;2.000;17.000', '16/12/2006;17:27:00;5.388;0.502;233.740;23.000;0.000;1.000;17.000', '16/12/2006;17:28:00;3.666;0.528;235.680;15.800;0.000;1.000;17.000']

#### Con el objeto sin datos perdidos, genere un objeto RDD donde los registro tengan la siguiente estructura:
<code>
    [(2006, 12, 18, 4.464, 0.136, 234.66, 19.0, 0.0, 37.0),
        (2006, 12, 18, 1.944, 0.084, 236.56, 8.2, 0.0, 2.0),
        (2006, 12, 9, 3.706, 0.062, 237.81, 15.6, 1.0, 38.0),
        (2006, 12, 1, 0.204, 0.0, 244.48, 0.8, 0.0, 0.0),
        (2006, 12, 10, 1.34, 0.092, 238.72, 5.6, 0.0, 0.0)]
</code>
<p>
    donde cada registro tendrá la siguiente composición: (año, mes, dia, global_activity_power,
global_reactive_power, voltage, global_intensity, submetering_1, submetering_2,
submetering_3)
</p>

In [28]:
hpc_proc = hpc_data\
            .filter(lambda x: x != header)\
            .filter(lambda x: '?' not in x)\
            .map(lambda x: x.split(';'))\
            .map(lambda x: (x[0].split('/')[2], x[0].split('/')[1], x[0].split('/')[0], 
                            x[2], x[3], x[4], x[5], x[6], x[7], x[8]))

hpc_proc.take(5)

[('2006', '12', '16', '4.216', '0.418', '234.840', '18.400', '0.000', '1.000', '17.000'), ('2006', '12', '16', '5.360', '0.436', '233.630', '23.000', '0.000', '1.000', '16.000'), ('2006', '12', '16', '5.374', '0.498', '233.290', '23.000', '0.000', '2.000', '17.000'), ('2006', '12', '16', '5.388', '0.502', '233.740', '23.000', '0.000', '1.000', '17.000'), ('2006', '12', '16', '3.666', '0.528', '235.680', '15.800', '0.000', '1.000', '17.000')]

#### Por defecto todos los datos dentro de este objeto serán considerados como strings. Conviértalos al tipo de dato pertinente ( int o float ). Devuelva con collect las primeras 5 observaciones

In [29]:
hpc_proc = hpc_data\
            .filter(lambda x: x != header)\
            .filter(lambda x: '?' not in x)\
            .map(lambda x: x.split(';'))\
            .map(lambda x: (int(x[0].split('/')[2]), int(x[0].split('/')[1]), int(x[0].split('/')[0]), 
                        float(x[2]), float(x[3]), float(x[4]), float(x[5]), float(x[6]), float(x[7]), float(x[8])))

hpc_proc.take(5)

[(2006, 12, 16, 4.216, 0.418, 234.84, 18.4, 0.0, 1.0, 17.0), (2006, 12, 16, 5.36, 0.436, 233.63, 23.0, 0.0, 1.0, 16.0), (2006, 12, 16, 5.374, 0.498, 233.29, 23.0, 0.0, 2.0, 17.0), (2006, 12, 16, 5.388, 0.502, 233.74, 23.0, 0.0, 1.0, 17.0), (2006, 12, 16, 3.666, 0.528, 235.68, 15.8, 0.0, 1.0, 17.0)]

### Ejercicio 3: Patrones globales

#### Extraiga la media, desviación estándar e intervalos de confianza (a .5 desviaciones estándar) para las columnas global_activity_power , global_reactive_power , voltage y global_intensity . Reporte sus resultados.

In [30]:
def patrones_globales(variable, index):
    gap_data = hpc_proc\
                .map(lambda x: x[index])
    print("Variable | Media | Desviación Estándar | Intervalos de confianza")
    gap_mean = gap_data.mean()
    gap_std = gap_data.stdev()
    print("{} | {} | {} | [{}, {}]".format(variable, round(gap_mean, 3), round(gap_std, 3), round(gap_mean-0.5*gap_std, 3), round(gap_mean+0.5*gap_std,3)))

In [31]:
patrones_globales('global_activity_power', 3)
patrones_globales('global_reactive_power', 4)
patrones_globales('voltage', 5)
patrones_globales('global_intensity', 6)

Variable | Media | Desviación Estándar | Intervalos de confianza
global_activity_power | 1.092 | 1.057 | [0.563, 1.62]
Variable | Media | Desviación Estándar | Intervalos de confianza
global_reactive_power | 0.124 | 0.113 | [0.067, 0.18]
Variable | Media | Desviación Estándar | Intervalos de confianza
voltage | 240.84 | 3.24 | [239.22, 242.46]
Variable | Media | Desviación Estándar | Intervalos de confianza
global_intensity | 4.628 | 4.444 | [2.406, 6.85]

### Ejercicio 4: Patrón temporal

#### Reporte el promedio de global_activity_power a nivel mensual y anual. Genere un objeto con la acción collectAsMap() del RDD procesado.

In [33]:
patron_temporal_data = hpc_proc\
                        .keyBy(lambda x: (x[0], x[1]))\
                        .mapValues(lambda x: (x[3], 1))\
                        .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
                        .mapValues(lambda x: (x[0]/x[1]))
patron_temporal_data.collectAsMap()

{(2006, 12): 1.9012951073117563, (2007, 1): 1.546033917290203, (2007, 3): 1.3186270301754266, (2007, 5): 0.9858618279569635, (2007, 7): 0.6673668246871165, (2007, 9): 0.9693181628779088, (2007, 11): 1.2944729848604817, (2008, 2): 1.181384486433406, (2008, 4): 1.1159720370370139, (2008, 6): 0.9940964374174476, (2008, 8): 0.27648819391553775, (2008, 10): 1.1367683924926801, (2008, 12): 1.2751893019811829, (2009, 1): 1.4102020206545822, (2009, 3): 1.2267347551413885, (2009, 5): 1.0128557666457598, (2009, 7): 0.6181208889685744, (2009, 9): 0.9868408454290529, (2009, 11): 1.2747434431353108, (2010, 2): 1.3758546058832082, (2010, 4): 1.0272951688695853, (2010, 6): 0.9696148895782476, (2010, 8): 0.5907781579088224, (2010, 10): 1.1633986872466457, (2007, 2): 1.401083535889756, (2007, 4): 0.8911889454619564, (2007, 6): 0.8268143754199269, (2007, 8): 0.764186202877764, (2007, 10): 1.1039107934942587, (2007, 12): 1.6264739353480613, (2008, 1): 1.4599202939134583, (2008, 3): 1.245336902708509, (20

### Ejercicio 5: Consumo energético general

El siguiente objetivo es calcular el consumo energético general para cada registro en el RDD de acuerdo a formula especificada.
* Implemente la función para todos los registros.
* Genere el RDD con el consumo energético general promedio por Año, Mes y Día.
* Reporte el consumo energético general para todos los días de Octubre del 2008.

In [38]:
def calc_consumo_enegergetico(x):
    return (x[3]*1000)/60 - x[7] - x[8] - x[9]

consumo_energetico_data = hpc_proc\
                            .keyBy(lambda x: (x[0], x[1], x[2]))\
                            .mapValues(lambda x: (calc_consumo_enegergetico(x), 1))\
                            .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
                            .mapValues(lambda x: (x[0]/x[1]))
consumo_energetico_data.take(5)

[((2006, 12, 17), 25.65740740740744), ((2006, 12, 19), 9.119374999999998), ((2006, 12, 21), 9.597635605006957), ((2006, 12, 23), 42.87252314814816), ((2006, 12, 25), 21.765046296296305)]

In [41]:
consumo_energetico_data = consumo_energetico_data\
                            .filter(lambda x: x[0][0] == 2008 and x[0][1] == 10)
consumo_energetico_data.collectAsMap()

{(2008, 10, 1): 8.57574074074073, (2008, 10, 3): 8.53773148148149, (2008, 10, 5): 10.773958333333315, (2008, 10, 7): 12.406342592592578, (2008, 10, 9): 12.023171296296297, (2008, 10, 11): 8.313263888888887, (2008, 10, 13): 9.243935185185197, (2008, 10, 15): 11.752962962962973, (2008, 10, 17): 11.286643518518511, (2008, 10, 19): 13.581643518518502, (2008, 10, 21): 13.572407407407413, (2008, 10, 23): 12.271689814814822, (2008, 10, 25): 7.746552135528486, (2008, 10, 27): 4.418425925925917, (2008, 10, 29): 14.916157407407386, (2008, 10, 31): 7.385787037037059, (2008, 10, 2): 9.024560185185186, (2008, 10, 4): 12.739074074074084, (2008, 10, 6): 8.952569444444434, (2008, 10, 8): 9.159444444444452, (2008, 10, 10): 8.937939814814817, (2008, 10, 12): 11.281249999999998, (2008, 10, 14): 10.596944444444446, (2008, 10, 16): 11.51157407407408, (2008, 10, 18): 15.777662037037043, (2008, 10, 20): 9.601087962962959, (2008, 10, 22): 10.932291666666671, (2008, 10, 24): 10.485949074074057, (2008, 10, 26):